# Neural Network for sentiment analysis of press texts

### Goal:
Creating a Neural Network for sentiment analysis of press materials

### Data:
Training set contains x news articles from 7 polish news portals. For each of these articles I calculate emotional score based on emotional valuation dictionary from Słowosieć - Polish Wordnet. Later the scores are converted to labels for positively and negatively valued articles, which will serve as a target in supervised learning

### Method choice
I want to use a Neural Network for this task, as it has higher potential for learning relationship between the words. Learning these relationships, which isn't possible with methods like Logistic Regression etc guarantees that the model will generalize better on texts with different topics than those in training set. 

In [3]:
# basic libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# NLP
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS 
import spacy
from collections import Counter
import nltk
from nltk.collocations import *

In [ ]:
!pip install
!python -m spacy download pl_core_news_lg

     |████████████████████████████████| 12.7MB 220kB/s 
     |████████████████████████████████| 1.1MB 43.2MB/s 
     |████████████████████████████████| 9.1MB 37.2MB/s 
     |████████████████████████████████| 460kB 36.2MB/s 
     |████████████████████████████████| 122kB 38.3MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=dd4c78f85be77dbf280b0671b4d6674049bc67d30e40496c7a51f417e7bab07a
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: smart-open 4.2.0
    Uninstallin

2021-03-18 09:22:04.297883: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 612.6MB 31kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_lg')


# Data preparation

I need to compute the emotional scores for each article in the set. In the emotional dictionary words are written as lemmas, and Logistic Regression(my baseline model) works better with lemmatized words, so I need to lemmatize the articles. Moreover, later in the notebook I need word vectors for creating the embedding matrix for Neural Net. Both of these processes can be done with SpaCy model, which I will use for preprocessing.

In [ ]:
# reading datasets
data = {"gazeta_pl": pd.read_csv("gazeta_pl.csv",sep="^"),
       "oko_press": pd.read_csv("OKO_press_all.csv",sep="^"),
       "onet": pd.read_csv("Onet.csv",sep="^",error_bad_lines=False),
       "wp":pd.read_csv("wp.csv",sep="^",error_bad_lines=False),
       "wPolityce":pd.read_csv("wPolityce.csv",sep="^"),
       "niezalezna":pd.read_csv("Niezalezna.csv",sep="^").rename({"Content\r":"Content"},axis=1),
       "krypol":pd.read_csv("krypol.csv",sep="^")}

b'Skipping line 1195: expected 5 fields, saw 9\n'


In [ ]:
# loading large polish spacy model
nlp = spacy.load("pl_core_news_lg")

In [ ]:
# checking wether model loaded properly
nlp("ja")[0].lemma_

'ja'

In [ ]:
# loading emotional dictionary
emotional_df = pd.read_csv(r"emotions.csv")

In [ ]:
def count_emotional_value(text,emotional_df):
    """
    Function calculates the emotional score of a single text based on evaluations provided by emotional valuation dictionary.
    text - lemmatized version of the text to calculate score on
    emotional_df - emotional dictionary read as pd.DataFrame
    """
    # variable storing emotional score
    emotional_score = 0
    # dictionary for translating emotional values into numerical values
    values = {"- m":-4,
             "- s":-1,
             "amb":0,
             "+ s":1,
             "+ m":4}
             
    # iterating over words in texts, checking wether word has emotional valuations, adding values to emotional score
    for word in text:
        word = str(word)
        if word in emotional_df.lemat.tolist():
            row = emotional_df[emotional_df["lemat"] == word]
            value = values[row.stopien_nacechowania.iloc[0]]
            emotional_score += value
            
    return emotional_score

In [ ]:
def preprocess_data(data,nlp,emotional_df):
    
    # process texts - saving columns with spacy docs and with lemmatized texts as lists of words
    data["processed"] = [nlp(x) for x in data.Content.astype("str")]
    data["lemmatized"] = [[x.lemma_ for x in text] for text in data.processed]
        
    
    
    # computing emotional values of texts
    data["emotional_value"] = [count_emotional_value(x,emotional_df) for x in data.lemmatized]
    
    
    return data

In [ ]:
# preprocessing data from each data frame

for key in data.keys():
        data[key] = preprocess_data(data[key],nlp,emotional_df)

In [ ]:
# checking distributions of positive and negative class
for key in data.keys():
    # print amounts of articles in positive and negative class
    print("Positive",data[key][data[key].emotional_value > 0].Title.count(),
          "Negative",data[key][data[key].emotional_value < 0].Title.count())
    
    # plot histogram to see the distribution of values
    plt.hist(data[key].emotional_value)
    plt.show()

# Data preparation for Neural Network

In the next steps I prepare the data to enter Neural Network - Train test split, Tokenization, Integer Encoding, creating starting embedding matrix, padding.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import train_test_split
# classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

from tensorflow.python.keras.utils import np_utils
# tensorflow and keras
import keras
from keras import regularizers, optimizers
from keras.layers.experimental.preprocessing import TextVectorization
from keras.layers import Activation, Bidirectional,Embedding, Dense, Dropout, Input, LSTM,MaxPooling1D,Flatten,SpatialDropout1D,BatchNormalization, Conv1D
from keras.models import Sequential
from keras.initializers import Constant
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import History,EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
# creating lerning set

learning_set = pd.DataFrame({"Emotion":[], "Text":[]})

for key in data.keys():
    # for each dataframe take emotional values and text, assign to new DF, append to learning set
    
    frame = pd.DataFrame({"Emotion":data[key].emotional_value,"Text":data[key].processed})
    learning_set = learning_set.append(frame,ignore_index=True)

In [ ]:
# creating learning set for logistic regression - lemmatized texs as strings
learning_set["Logistic"] = [str([x.lemma_ for x in text]).replace("'","").replace(",","").replace("[","").replace("]","")\
                            for text in learning_set.Text]

In [4]:
learning_set = pd.read_csv("learning_set.csv",sep="^")

In [5]:
# drop NaNs from learning set
learning_set.dropna(inplace=True)

In [6]:
# prepare target and features variable for Neural Network and Logistic Regression

y = pd.Series([0 if x<0 else 1 for x in learning_set["Emotion"]])
y_log = pd.Series([0 if x<0 else 1 for x in learning_set["Emotion"]])
X_log = pd.Series(learning_set["Logistic"])
X = pd.Series(learning_set["Text"])

In [7]:
#  train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
X_train_l,X_test_l,y_train_l,y_test_l = train_test_split(X_log,y_log,test_size=0.2)

Baseline model - logistic regression

In [8]:
# fit the data on simple logistic regression to check wether the set is classifiable

mdl = make_pipeline(CountVectorizer(),LogisticRegression(solver="liblinear"))
                    
mdl.fit(X_train_l,y_train_l)
mdl.score(X_test_l,y_test_l)

0.8435434111943587

In [9]:
# prepare train and test variables for tokenizing - make sure that every element is a string

X_train_for_tokenizer = [[str(x) for x in y] for y in X_train] 
X_test_for_tokenizer = [[str(x) for x in y] for y in X_test] 



In [10]:
X_train[:10]

9299     ['Pytany', 'w', 'programie', 'Michała', 'Racho...
5240     ['Kolejna', 'partia', 'szczepionek', 'na', 'CO...
5025     ['Dopiero', 'zaczynało', 'świtać', ',', 'gdy',...
403      ['Minionej', 'doby', 'odnotowano', '7', '038',...
10130    ['81', '-', 'letnia', 'mieszkanka', 'Olsztyna'...
378      ['Kancelaria', 'Prezydenta', 'poinformowała', ...
8935     ['Jeszcze', 'do', '25', 'lutego', 'uczniowie',...
319      ['Amerykańska', 'agencja', 'kosmiczna', 'NASA'...
3976     ['Ojciec', 'Święty', 'nawiązał', 'do', '90', '...
10830    ['Politycy', ',', 'publicyści', 'oraz', 'dyplo...
Name: Text, dtype: object

In [11]:
# median length of textsw
np.median([len(x) for x in X_train])

2887.0

In [12]:
# tokenizer with custom filter - does not filter out comas, stops, ? and !, as they can carry important insights
t = Tokenizer(num_words=50000,lower=False,filters='#$%&+-/<=>@[\\]^_`{|}~\t\n')
t.fit_on_texts(X_train)

# vocab size for embedding martix
vocab_size = len(t.word_index) + 1

# integer encode the documents
encoded_docs = t.texts_to_sequences(X_train)
encoded_test = t.texts_to_sequences(X_test)

# padding texts 
max_length = 1000
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

    


In [13]:
padded_docs[0]

array([ 1606,     3,   863,  1775, 11104,    13,     4,     5, 10257,
          13,    16,    14,     1,    42,   546,  6051,  5902,     7,
         415,   824,     8,   121,     1, 10025,   527,     1, 13247,
           1,   740,  4517, 10792,   291,     1,    10,     3,  3548,
       22802,     1, 22803,     1,    15,    38,    11,   610,  6403,
       25853,     2,     4,     5,  4677,    12,    58,   658,     2,
         757,    31,  1042,  3121,  2057,  1594,     3,   666,  1419,
           3,   273,    59,     2,     6,    14,    12,    15,  3121,
        1268,     4,     5,   761,     2,    18,   334,  1078,  1024,
           6,  1274,   730,  7778,  6404,   509,  1326,    25,  1326,
           4,     5,   659,  1021,  2039,     3,   736,    11,  9320,
        6590,  5961,    27,   357,   263,     1,   159,   509,   393,
           1,  2907,     1,   546,     6,   567,   123,   437,   919,
         527,     2,   143,  6793,  5025, 46875,  7505,  7240, 25854,
          30,  4741,

# Creating architecture and training model

I use architecture based on LSTM's, as they can work with text data as sequences and they capture relationship between words best. I use LSTM's instead of simple RNN's because i want the model to consider long term relationships between words, which simple RNN's do worse. 





In [27]:
history = History()

# Creating 
model = Sequential()
model.add(Embedding(
    50000,
    100,
    input_length = max_length,
    trainable=True))
model.add(Bidirectional(LSTM(32,return_sequences=True)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.65))


model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.65))
model.add(Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1000, 100)         5000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 64)          34048     
_________________________________________________________________
batch_normalization_15 (Batc (None, 1000, 64)          256       
_________________________________________________________________
activation_15 (Activation)   (None, 1000, 64)          0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 1000, 64)          0         
_________________________________________________________________
dense_12 (Dense)             (None, 1000, 32)          2080      
_________________________________________________________________
batch_normalization_16 (Batc (None, 1000, 32)         

In [28]:
Adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss="binary_crossentropy",optimizer=Adam, metrics=["accuracy"])
save_best_model = ModelCheckpoint("sentiment.h5",save_best_only=True)

model.fit(padded_docs,y_train,validation_split=0.2,batch_size=32,epochs=15,callbacks=[history,save_best_model])


Epoch 1/15
227/227 [==============================] - 30s 116ms/step - loss: 0.8243 - accuracy: 0.5050 - val_loss: 0.6435 - val_accuracy: 0.6942
Epoch 2/15
227/227 [==============================] - 25s 111ms/step - loss: 0.6551 - accuracy: 0.6178 - val_loss: 0.6220 - val_accuracy: 0.6554
Epoch 3/15
227/227 [==============================] - 25s 112ms/step - loss: 0.5469 - accuracy: 0.7442 - val_loss: 0.6292 - val_accuracy: 0.6617
Epoch 4/15
227/227 [==============================] - 25s 111ms/step - loss: 0.4859 - accuracy: 0.7960 - val_loss: 0.6690 - val_accuracy: 0.6660
Epoch 5/15
227/227 [==============================] - 25s 110ms/step - loss: 0.4636 - accuracy: 0.8222 - val_loss: 0.7916 - val_accuracy: 0.6031
Epoch 6/15
227/227 [==============================] - 25s 111ms/step - loss: 0.4531 - accuracy: 0.8277 - val_loss: 0.6338 - val_accuracy: 0.6764
Epoch 7/15
227/227 [==============================] - 25s 112ms/step - loss: 0.3674 - accuracy: 0.8777 - val_loss: 0.7307 - val_ac

KeyboardInterrupt: ignored